In [45]:
import numpy as np
import pandas as pd
import pickle

In [46]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [47]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 150)

In [48]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset()
df_all, df_census = utils.merge_dataset(df_train, df_test, pop=False, unemploy=False, census=True, coord=True, fix_pop=True, outlier=True)

smooth_outlier: max_scale=40
# of fixed cfips: 559
# of fixed value: 925


In [49]:
params={
        "act_thre": 140,
        "abs_thre": 0,
        "USE_LAG": 7,
        "USE_TREND": False,
        "USE_TREND_MULTI": True,
        "blacklist": [],
        "blacklistcfips": [],
        "clip": (-0.0044, 0.0046),
        "model": 'ensemble'
    }

## バリデーション

### 前の予測結果を用いて予測していく

In [50]:
# accum_pred = models.LgbmBaseline('params_accum',df_subm, df_all, df_census, start_all_dict=32, params=params)
# accum_pred.accum_validation(m_len=5, max_pred_m=1)

## Submissionの作成

### 予測結果を活用して予測

In [51]:
test_subm = models.LgbmBaseline('test_subm_accum', df_subm, df_all, df_census, start_all_dict=40, params=params)
test_subm.create_submission(accum=True, target_scale=[41])

add lag features: max_scale=40
created df_all_dict[40]
valid_times:  41
pred_m:  1
train_times:  40
use df_all_dict[40]
['scale', 'state_i', 'pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017', 'pct_college_2018', 'pct_college_2019', 'pct_college_2020', 'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018', 'pct_foreign_born_2019', 'pct_foreign_born_2020', 'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018', 'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021', 'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019', 'median_hh_inc_2020', 'median_hh_inc_2021', 'lng', 'lat', 'select_rate1_lag1', 'select_rate1_lag2', 'select_rate1_lag3', 'select_rate1_lag4', 'select_rate1_lag5', 'select_rate1_lag6', 'select_rate1_lag7', 'select_rate1_rsum2', 'select_rate1_rsum4', 'select_rate1_rsum6', 'select_rate1_rsum8', 'select_rate1_rsum10', 'select_active_lag1_diff1', 'select_active_lag1_diff2', 'selec

In [58]:
df_submission = pd.read_csv('../submission/test_subm_accum_2023-02-25_21:02:46.csv', index_col='row_id')
df_merged = utils.compare_submission(df_submission, 'submission_19346')

In [59]:
df_merged['smape'].mean()

0.009844264000484165

In [57]:
# ensemble
0.01261112205424169

# trend_multi有り
0.03200090765098272

0.03200090765098272